In [1]:
import argparse
import csv
import gzip
from typing import List
from ast import literal_eval

import numpy as np
import pandas as pd
from pandas.api.types import is_object_dtype
from pprint import pprint
from tqdm import tqdm

## Transform Categories 

In [2]:
read_path = '../data/interim/meta_Electronics.csv'
META_COLS = ['asin', 
             'category', 
             'title', 
             'description', 
             'price', 
             'brand', 
             'also_buy', 
             'also_view',
             'similar_item']
df = pd.read_csv(read_path, usecols = META_COLS, converters = {'also_buy' : literal_eval,
                                                              'also_view' : literal_eval, 
                                                              'category' : literal_eval,
                                                              'description' : literal_eval,})


KeyboardInterrupt



In [ ]:
df.head()

In [ ]:
def get_category_lvl(category_list: list, lvl = 0) -> str:
    try:
        return category_list[lvl]
    except IndexError:
        return 'NA_VALUE'

In [ ]:
df['category_lvl_1'] = df['category'].apply(get_category_lvl, args=(0,))
df['category_lvl_2'] = df['category'].apply(get_category_lvl, args=(1,))
df['category_lvl_3'] = df['category'].apply(get_category_lvl, args=(2,))
df['category_lvl_4'] = df['category'].apply(get_category_lvl, args=(3,))

In [ ]:
df.head()

In [ ]:
df['similar_item'].unique()[2]

In [ ]:
df['similar_item'] = np.where(df['similar_item'].isnull(), 0, 1)

In [ ]:
write_path = '../data/interim/meta_Electronics_transformed.csv'
df.to_csv(write_path, index = False)

## Node Relationships

In [4]:
read_path = '../data/interim/meta_Electronics_transformed.csv'
df = pd.read_csv(read_path, on_bad_lines = 'warn',
                 dtype = {'asin' : 'str', 
                          'title' : 'str',
                          'brand' : 'str'},
                 converters = {'also_buy' : literal_eval,
                               'also_view' : literal_eval, 
                               'category' : literal_eval,
                               'description' : literal_eval})

/tmp/ipykernel_12399/3719423463.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(read_path, on_bad_lines = 'warn',


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786445 entries, 0 to 786444
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   category        786445 non-null  object
 1   description     786445 non-null  object
 2   title           786426 non-null  object
 3   also_buy        786445 non-null  object
 4   brand           781009 non-null  object
 5   also_view       786445 non-null  object
 6   similar_item    786445 non-null  int64 
 7   price           310336 non-null  object
 8   asin            786445 non-null  object
 9   category_lvl_1  786445 non-null  object
 10  category_lvl_2  786445 non-null  object
 11  category_lvl_3  786445 non-null  object
 12  category_lvl_4  786445 non-null  object
dtypes: int64(1), object(12)
memory usage: 78.0+ MB


In [6]:
RELATED_FIELDS = ['also_buy', 'also_view']
df['is_na'] = df.apply(lambda x : not bool(sum([len(x[col]) for col in RELATED_FIELDS])), axis = 1)

In [7]:
df.head()

,category,description,title,also_buy,brand,also_view,similar_item,price,asin,category_lvl_1,category_lvl_2,category_lvl_3,category_lvl_4,is_na
0,"[electronics, camera &amp; photo, video survei...",[the following camera brands and models have b...,genuine geovision 1 channel 3rd party nvr ip s...,[],geovision,[],0,$65.00,0011300000,electronics,camera &amp; photo,video surveillance,surveillance systems,True
1,"[electronics, camera &amp; photo]",[this second edition of the handbook of astron...,"books ""handbook of astronomical image processi...",[0999470906],33 books co.,"[0943396670, 1138055360, 0999470906]",0,NaN,0043396828,electronics,camera &amp; photo,NA_VALUE,NA_VALUE,False
2,"[electronics, ebook readers &amp; accessories,...",[a zesty tale. (publishers weekly)<br /><br />...,one hot summer,"[0425167798, 039914157x]",visit amazon's carolina garcia aguilera page,[],0,$11.49,0060009810,electronics,ebook readers &amp; accessories,ebook readers,NA_VALUE,False
3,"[electronics, ebook readers & accessories, ebo...",[],hurray for hattie rabbit: story and pictures (...,"[0060219521, 0060219580, 0060219394]",visit amazon's dick gackenbach page,"[0060219521, 0060219475, 0060219394]",0,.a-section.a-spacing-mini{margin-bottom:6px!im...,0060219602,electronics,ebook readers & accessories,ebook readers,NA_VALUE,False
4,"[electronics, ebook readers & accessories, ebo...",[&#8220;sex.lies.murder.fame. is brillllli&#82...,sex.lies.murder.fame.: a novel,[],visit amazon's lolita files page,[],0,$13.95,0060786817,electronics,ebook readers & accessories,ebook readers,NA_VALUE,True


In [8]:
df = df[~df['is_na']]
df = df[~df['title'].isnull()]
df = df[['asin'] + RELATED_FIELDS]

In [9]:
df

,asin,also_buy,also_view
1,0043396828,[0999470906],"[0943396670, 1138055360, 0999470906]"
2,0060009810,"[0425167798, 039914157x]",[]
3,0060219602,"[0060219521, 0060219580, 0060219394]","[0060219521, 0060219475, 0060219394]"
5,0070524076,"[0073049557, 0134454170, 1118142063, 007733968...","[0073512141, 0077339681, 0073049557, 007304956..."
6,0091912407,[0330509691],[b0719ldqr1]
...,...,...,...
786424,b01hjcn5gc,"[b01hjcn55i, b01d6ieg6q]",[]
786431,b01hjffhtc,[],"[b07bc99b8l, b06x9k9x1g, b07gdc29sr, b07cwng32..."
786433,b01hjfrhya,[b01m096i7q],[]
786437,b01hjf704m,[],"[b01ccmun8c, b003mttjoy, b008ifxqfu, b00jdvrci..."


In [10]:
df['also_bought_count'] = df['also_buy'].apply(len)

In [11]:
df = df[df['also_bought_count'] >= 2]
df

,asin,also_buy,also_view,also_bought_count
2,0060009810,"[0425167798, 039914157x]",[],2
3,0060219602,"[0060219521, 0060219580, 0060219394]","[0060219521, 0060219475, 0060219394]",3
5,0070524076,"[0073049557, 0134454170, 1118142063, 007733968...","[0073512141, 0077339681, 0073049557, 007304956...",9
7,0101635370,"[b01naj3kqb, b00wyspt0c, b00af40u5g, b00ofvnm4...","[b01naj3kqb, b00ofvnm4g, b00l41wy8k, b07f34pnp...",5
10,0151004714,"[0307596907, 030726419x, 0140140999]","[0393356051, 0307743764, 0393355098, 030727666...",3
...,...,...,...,...
786418,b01hjcn5to,"[b01hjcn55i, b01d6ieg6q, b008plzaba]","[b01hrva9b4, b01hrvabd0, b01hjcn55i, b01f9rh5m4]",3
786420,b01hjdr9dq,"[b01czelia4, b06xccxmtb, b00ix1kebs, b01ey0x6f...","[b004wi1so8, b077ly5rqh, b00hkvqujw, b07548g6z...",11
786421,b01hjdxfqq,"[b002q907ew, b00hsf65mc, b009vcz4v8]",[],3
786423,b01hjdnl60,"[b00ivpu786, b01dob6y5q, b010q57t02, b01m592j9...","[b07dvf9lkg, b07cpsbxwl, b07d7xbk4k, b07cyw6jx...",10


In [12]:
df.drop(columns = ['also_bought_count'], inplace=True)

/tmp/ipykernel_12399/1718674056.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns = ['also_bought_count'], inplace=True)


In [13]:
def explode_on_related(df: pd.DataFrame, relationship : str) -> pd.DataFrame:
    vals = df[relationship].values.tolist()
    lens = [len(val_list) for val_list in vals]
    vals_array = np.repeat(df['asin'], lens)
    exploded_df = pd.DataFrame(np.column_stack((vals_array, np.concatenate(vals))),
                               columns=['asin', 'related'])
    exploded_df['relationship'] = relationship
    return exploded_df

In [14]:
also_buy_df = explode_on_related(df, 'also_buy')
also_view_df = explode_on_related(df, 'also_view')

In [15]:
also_buy_df

,asin,related,relationship
0,0060009810,0425167798,also_buy
1,0060009810,039914157x,also_buy
2,0060219602,0060219521,also_buy
3,0060219602,0060219580,also_buy
4,0060219602,0060219394,also_buy
...,...,...,...
1919007,b01hjdnl60,b01mqmjfdk,also_buy
1919008,b01hjdnl60,b06xx29s9q,also_buy
1919009,b01hjdnl60,b00j4eqvpg,also_buy
1919010,b01hjcn5gc,b01hjcn55i,also_buy


In [16]:
also_view_df

,asin,related,relationship
0,0060219602,0060219521,also_view
1,0060219602,0060219475,also_view
2,0060219602,0060219394,also_view
3,0070524076,0073512141,also_view
4,0070524076,0077339681,also_view
...,...,...,...
1208868,b01hjdnl60,b01cw4bi6g,also_view
1208869,b01hjdnl60,b06xfqtdql,also_view
1208870,b01hjdnl60,b0773fxvkl,also_view
1208871,b01hjdnl60,b07gv3nb69,also_view


In [17]:
combined_df = pd.concat([also_buy_df, also_view_df], axis=0)

In [18]:
write_path = '../data/interim/meta_Electronics_relationship.csv'
combined_df.to_csv(write_path, index = False)

## Prep Edges

In [3]:
read_path = '../data/interim/meta_Electronics_relationship.csv'
df = pd.read_csv(read_path)

In [4]:
df.head()

,asin,related,relationship
0,0060009810,0425167798,also_buy
1,0060009810,039914157x,also_buy
2,0060219602,0060219521,also_buy
3,0060219602,0060219580,also_buy
4,0060219602,0060219394,also_buy


In [5]:
sample_size = None
if sample_size:
    sample_idx = np.random.choice(df.shape[0], 
                                  size=sample_size,
                                  replace=False)
    df = df.iloc[sample_idx]

In [6]:
df.head()

,asin,related,relationship
0,0060009810,0425167798,also_buy
1,0060009810,039914157x,also_buy
2,0060219602,0060219521,also_buy
3,0060219602,0060219580,also_buy
4,0060219602,0060219394,also_buy


In [7]:
df.shape

(3127885, 3)

In [8]:
col_list = ['asin', 'related']
pairs = df[col_list].values
df['product_pair'] = ['|'.join(arr) for arr in pairs]

In [9]:
df

,asin,related,relationship,product_pair
0,0060009810,0425167798,also_buy,0060009810|0425167798
1,0060009810,039914157x,also_buy,0060009810|039914157x
2,0060219602,0060219521,also_buy,0060219602|0060219521
3,0060219602,0060219580,also_buy,0060219602|0060219580
4,0060219602,0060219394,also_buy,0060219602|0060219394
...,...,...,...,...
3127880,b01hjdnl60,b01cw4bi6g,also_view,b01hjdnl60|b01cw4bi6g
3127881,b01hjdnl60,b06xfqtdql,also_view,b01hjdnl60|b06xfqtdql
3127882,b01hjdnl60,b0773fxvkl,also_view,b01hjdnl60|b0773fxvkl
3127883,b01hjdnl60,b07gv3nb69,also_view,b01hjdnl60|b07gv3nb69


In [10]:
relationship_weights = {'also_buy' : 1.0,
                        'also_view': 0.5}

In [11]:
df['weight'] = 0
for relationship, weight in relationship_weights.items():
    print(relationship)
    df.loc[df['relationship'] == relationship, 'weight'] += weight
df.head()

also_buy
also_view


,asin,related,relationship,product_pair,weight
0,0060009810,0425167798,also_buy,0060009810|0425167798,1.0
1,0060009810,039914157x,also_buy,0060009810|039914157x,1.0
2,0060219602,0060219521,also_buy,0060219602|0060219521,1.0
3,0060219602,0060219580,also_buy,0060219602|0060219580,1.0
4,0060219602,0060219394,also_buy,0060219602|0060219394,1.0


In [12]:
print(f'Original NUmber of edges = {df.shape[0]}')

Original NUmber of edges = 3127885


In [13]:
df = df.groupby('product_pair').agg({'weight':'sum'}).reset_index()

In [14]:
print(f'Duplicated no. of edges = {df.shape[0]}')

Duplicated no. of edges = 2881996


In [15]:
def split_product_pair(product_pair):
    result = product_pair.split('|')
    return result[0], result[1]

In [16]:
df['product1'], df['product2'] = zip(*df['product_pair'].apply(split_product_pair))

In [17]:
df.head()

,product_pair,weight,product1,product2
0,0060009810|039914157x,1.0,0060009810,039914157x
1,0060009810|0425167798,1.0,0060009810,0425167798
2,0060219602|0060219394,1.5,0060219602,0060219394
3,0060219602|0060219475,0.5,0060219602,0060219475
4,0060219602|0060219521,1.5,0060219602,0060219521


In [18]:
df = df[['product1', 'product2', 'weight', 'product_pair']]

In [19]:
if sample_size:
    write_path = '../data/interim/meta_Electronics_edges_sample.csv'
else:
    write_path = '../data/interim/meta_Electronics_edges.csv'

df.to_csv(write_path, index = False)